In [16]:
import numpy as np
from scipy import sparse
from helperfunctions import user_to_index, index_to_user, subreddit_to_index, index_to_subreddit
from massdiff_utils import parseDevSet, createDictionaries, createA, createR
import time

In [8]:
infile = 'inputGraph1.txt'
devSet = parseDevSet('devUsers.txt')
users_to_id,subs_to_id,id_to_users,id_to_subs = createDictionaries(infile)

creating dictionaries
reading file
done creating dictionaries


In [9]:
subredditUser, A_col= createA(infile, users_to_id, subs_to_id)
subredditDeg = sparse.csr_matrix(A_col).sum(axis=1) #degree (subreddit x 1) 


creating A
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [10]:
R_col = createR(infile, users_to_id, subs_to_id)


creating R
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [66]:
total = 0
zk = 0
mrr = 0
map_ = 0
#outfile = 'md.txt'
#f= open(outfile,"w")

for key in devSet.keys():
    #f.write(key + ' ')
    print(zk) 
    userId = users_to_id[key]
    user = id_to_users[userId]
    oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
    connected_users = set()
    for subredditId in oldSubreddits:
        connected_users.update(subredditUser[subredditId])    
    connected_users=list(connected_users)
    devSet[user]['connectedUsers'] = connected_users
    
    i = A_col[:,userId].multiply(np.reciprocal(subredditDeg.astype(float)))
    j = A_col[:,connected_users]
    fij = i.T.dot(j)
    jp = j.multiply(np.reciprocal(j.sum(axis=0).astype(float)))
    user_i = jp.dot(fij.T)
    top20 = np.argsort(user_i.toarray(),axis=0)[::-1]
    
    '''
    for t in top20:
        print(id_to_subs[t[0]])
        #f.write(str(id_to_subs[t[0]]) + ' ')
    #f.write('\n')
    '''
    #precision @ k
    ls = []
    k = 10
    i = p = pk = 0
    while i<k:
        if top20[p][0] not in oldSubreddits:
            ls.append(id_to_subs[top20[p][0]])
            if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
                pk +=1
            i +=1
        p+=1
    total +=pk/float(k)
    '''
    #mrr
    i = 1
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                mrr += 1./i
                break
            i+=1
    
    #map 
    i = 1
    correctSoFar = 0
    actual = float(len(devSet[user]['newSubreddits']))
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                correctSoFar+=1
                map_ += correctSoFar/actual/i
            i+=1

    zk+=1
    '''
#f.close()

#print(time)
print ls

0
['t5_2qh1i', 't5_2z47n', 't5_3csff', 't5_2qh03', 't5_2se9w', 't5_2yt52', 't5_3ggz0', 't5_3h47q', 't5_2ukac', 't5_2qh33']


In [50]:
total = 0
zk = 0
mrr = 0
map_ = 0

key = 'desertfreeze'
lamba = 0


userId = users_to_id[key]
user = id_to_users[userId]
oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
connected_users = set()
for subredditId in oldSubreddits:
    connected_users.update(subredditUser[subredditId])    
connected_users=list(connected_users)
devSet[user]['connectedUsers'] = connected_users

i = A_col[:,userId].multiply(np.reciprocal(subredditDeg.astype(float)))
j = A_col[:,connected_users]
fij = i.T.dot(j)
jp = j.multiply(np.reciprocal(j.sum(axis=0).astype(float)))
user_i = jp.dot(fij.T)
top20 = np.argsort(user_i.toarray(),axis=0)[::-1]


for i in range(10):
    print id_to_subs[top20[i][0]]

#precision @ k
k = 10
i = p = pk = 0
while i<k:
    if top20[p][0] not in oldSubreddits:
        if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
            pk +=1
        i +=1
    p+=1

total +=pk/float(k)

t5_2qh1i
t5_2qh33
t5_2qh0u
t5_2qh03
t5_2qqjc
t5_2qt55
t5_2qh13
t5_2qh1e
t5_2qh3l
t5_2qh1o


In [51]:
total = 0
zk = 0
mrr = 0
map_ = 0
key = "desertfreeze"
lamba = 0

userId = users_to_id[key]
user = id_to_users[userId]
oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
connected_users = set()
for subredditId in oldSubreddits:
    connected_users.update(subredditUser[subredditId])    
connected_users=list(connected_users)
devSet[user]['connectedUsers'] = connected_users

R_i = R_col[:,userId]
R_cu = R_col[:,connected_users]
R_i_sqrt = np.power(R_i.power(2).sum(axis=0),0.5)
R_cu_sqrt = np.power(R_cu.multiply(R_cu).sum(axis=0),0.5)
cos_i = (R_i.T.dot(R_cu).multiply(np.reciprocal(R_cu_sqrt))).multiply(np.reciprocal(R_i_sqrt))
#cos_i = Cos[:,userId]

cos_i = cos_i.T
j = A_col[:,connected_users]
denom = j.dot(cos_i)

i = A_col[:,userId].multiply(np.reciprocal(denom.astype(float).toarray()))

ij = i.T.dot(j)
fij = ij.multiply(cos_i.T)


jp = j.multiply(np.reciprocal(np.power(j.sum(axis=0).astype(float),1-lamba)))
user_i = jp.dot(fij.T).multiply(np.power(subredditDeg,lamba))
top20 = np.argsort(user_i.toarray(),axis=0)[::-1]


for i in range(10):
    print id_to_subs[top20[i][0]]

#precision @ k
k = 10
i = p = pk = 0
while i<k:
    if top20[p][0] not in oldSubreddits:
        if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
            pk +=1
        i +=1
    p+=1

total +=pk/float(k)

#mrr
i = 1
for t in top20:
    if t[0] not in oldSubreddits:
        if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
            mrr += 1./i
            break
        i+=1

#map 
i = 1
correctSoFar = 0
actual = float(len(devSet[user]['newSubreddits']))
for t in top20:
    if t[0] not in oldSubreddits:
        if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
            correctSoFar+=1
            map_ += correctSoFar/actual/i
        i+=1

print(total)
print(mrr)
print(map_)
#print(time)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in reciprocal


t5_2qq5c
t5_300m6
t5_32di1
t5_2sc2j
t5_2wm0g
t5_2u2ey
t5_2r5a3
t5_2qoil
t5_2t7ax
t5_2vplj
0.2
0.25
0.118589992103


In [26]:
total = 0
zk = 0
mrr = 0
map_ = 0
k=10.0
lamba = 0.5

for key in devSet.keys():
    print(zk) 
    userId = users_to_id[key]
    user = id_to_users[userId]
    oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
    connected_users = set()
    for subredditId in oldSubreddits:
        connected_users.update(subredditUser[subredditId])    
    connected_users=list(connected_users)
    devSet[user]['connectedUsers'] = connected_users

    R_i = R_col[:,userId]
    R_cu = R_col[:,connected_users]
    R_i_sqrt = np.power(R_i.power(2).sum(axis=0),0.5)
    R_cu_sqrt = np.power(R_cu.multiply(R_cu).sum(axis=0),0.5)
    cos_i = (R_i.T.dot(R_cu).multiply(np.reciprocal(R_cu_sqrt))).multiply(np.reciprocal(R_i_sqrt))
    #cos_i = Cos[:,userId]
    
    cos_i = cos_i.T
    j = A_col[:,connected_users]
    denom = j.dot(cos_i)

    i = A_col[:,userId].multiply(np.reciprocal(denom.astype(float).toarray()))

    ij = i.T.dot(j)
    fij = ij.multiply(cos_i.T)

    
    jp = j.multiply(np.reciprocal(np.power(j.sum(axis=0).astype(float),1-lamba)))
    user_i = jp.dot(fij.T).multiply(np.power(subredditDeg,lamba))
    top20 = np.argsort(user_i.toarray(),axis=0)[::-1]

    '''#precision @ k
    k = 10
    i = p = pk = 0
    while i<k:
        if top20[p][0] not in oldSubreddits:
            if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
                pk +=1
            i +=1
        p+=1
    total +=pk/float(k)'''
    
    #mrr
    i = 1
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                mrr += 1./i
                break
            i+=1
    
    #map 
    i = 1
    correctSoFar = 0
    actual = float(len(devSet[user]['newSubreddits']))
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                correctSoFar+=1
                map_ += correctSoFar/actual/i
            i+=1

    zk+=1


print(total/100)
print(mrr/100)
print(map_/100)
#print(time)

0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in reciprocal


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
0.368367381598
0.089093510431


In [ ]:
mixed similarity (0.9)

In [27]:
total = 0
zk = 0
mrr = 0
map_ = 0
k=10.0
lamba = 0.9


for key in devSet.keys():
    print(zk)
    userId = users_to_id[key]
    user = id_to_users[userId]
    oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
    connected_users = set()
    for subredditId in oldSubreddits:
        connected_users.update(subredditUser[subredditId])    
    connected_users=list(connected_users)
    devSet[user]['connectedUsers'] = connected_users

    R_i = R_col[:,userId]
    R_cu = R_col[:,connected_users]
    R_i_sqrt = np.power(R_i.power(2).sum(axis=0),0.5)
    R_cu_sqrt = np.power(R_cu.multiply(R_cu).sum(axis=0),0.5)
    cos_i = (R_i.T.dot(R_cu).multiply(np.reciprocal(R_cu_sqrt))).multiply(np.reciprocal(R_i_sqrt))
    #cos_i = Cos[:,userId]
    
    cos_i = cos_i.T
    j = A_col[:,connected_users]
    denom = j.dot(cos_i)

    i = A_col[:,userId].multiply(np.reciprocal(denom.astype(float).toarray()))

    ij = i.T.dot(j)
    fij = ij.multiply(cos_i.T)

    
    jp = j.multiply(np.reciprocal(np.power(j.sum(axis=0).astype(float),1-lamba)))
    user_i = jp.dot(fij.T).multiply(np.power(subredditDeg,lamba))
    top20 = np.argsort(user_i.toarray(),axis=0)[::-1]

    '''#precision @ k
    k = 10
    i = p = pk = 0
    while i<k:
        if top20[p][0] not in oldSubreddits:
            if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
                pk +=1
            i +=1
        p+=1
    total +=pk/float(k)'''
    
    #mrr
    i = 1
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                mrr += 1./i
                break
            i+=1
    
    #map 
    i = 1
    correctSoFar = 0
    actual = float(len(devSet[user]['newSubreddits']))
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                correctSoFar+=1
                map_ += correctSoFar/actual/i
            i+=1

    zk+=1


print(total/100)
print(mrr/100)
print(map_/100)
#print(time)

0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in reciprocal


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0
0.374790345917
0.0860807674964


In [68]:
total = 0
zk = 0
mrr = 0
map_ = 0
k=10.0
lamba = 0
#outfile = 'msd.txt'
#f= open(outfile,"w")

for key in devSet.keys():
    #f.write(key + ' ')
    print(zk) 
    zk+=1
    userId = users_to_id[key]
    user = id_to_users[userId]
    oldSubreddits = [subs_to_id[u] for u in devSet[user]['oldSubreddits']]
    connected_users = set()
    for subredditId in oldSubreddits:
        connected_users.update(subredditUser[subredditId])    
    connected_users=list(connected_users)
    devSet[user]['connectedUsers'] = connected_users

    R_i = R_col[:,userId]
    R_cu = R_col[:,connected_users]
    R_i_sqrt = np.power(R_i.power(2).sum(axis=0),0.5)
    R_cu_sqrt = np.power(R_cu.multiply(R_cu).sum(axis=0),0.5)
    cos_i = (R_i.T.dot(R_cu).multiply(np.reciprocal(R_cu_sqrt))).multiply(np.reciprocal(R_i_sqrt))
    #cos_i = Cos[:,userId]
    
    cos_i = cos_i.T
    j = A_col[:,connected_users]
    denom = j.dot(cos_i)

    i = A_col[:,userId].multiply(np.reciprocal(denom.astype(float).toarray()))

    ij = i.T.dot(j)
    fij = ij.multiply(cos_i.T)

    
    jp = j.multiply(np.reciprocal(np.power(j.sum(axis=0).astype(float),1-lamba)))
    user_i = jp.dot(fij.T).multiply(np.power(subredditDeg,lamba))
    top20 = np.argsort(user_i.toarray(),axis=0)[::-1]
    
    
    ls = []
    '''
    for t in top20:
        print(str(id_to_subs[t[0]]) + ' ')
    '''
    #precision @ k
    
    k = 10
    i = p = pk = 0
    while i<k:
        if top20[p][0] not in oldSubreddits:
            if id_to_subs[top20[p][0]] in devSet[user]['newSubreddits']:
                pk +=1
            i +=1
            ls.append(id_to_subs[top20[p][0]])
        p+=1
    total +=pk/float(k)
    
    '''
    #mrr
    i = 1
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                mrr += 1./i
                break
            i+=1
    
    #map 
    i = 1
    correctSoFar = 0
    actual = float(len(devSet[user]['newSubreddits']))
    for t in top20:
        if t[0] not in oldSubreddits:
            if id_to_subs[t[0]] in devSet[user]['newSubreddits']:
                correctSoFar+=1
                map_ += correctSoFar/actual/i
            i+=1

    zk+=1
    '''
#f.close()

#print(time)
print ls
print total

0
['t5_3csff', 't5_2qh1i', 't5_3ggz0', 't5_2se9w', 't5_2z47n', 't5_2qh03', 't5_2yt52', 't5_2ukac', 't5_4bnoh', 't5_2sn6d']
0.3


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in reciprocal


In [ ]:
A_dok = sparse.dok_matrix((5,3))
A_dok[0,:] = A_dok[1,1] = A_dok[2,0] = A_dok[3,0] = A_dok[4,0] = A_dok[4,2] = A_dok[3,2] = 1 

In [58]:
devSet = parseDevSet('user1.txt')